# Document processing with Amazon Nova Understanding Models

In this notebook we will demonstrate how to use [Amazon Nova *understanding* models](https://aws.amazon.com/ai/generative-ai/nova/) to extract information directly from binary documents.

Among Amazon's Nova more interesting features is its document support, which means we no longer need to transform documents such as PDFs or Doc without the need to extract the document's text using an OCR first. See more at: [https://docs.aws.amazon.com/nova/latest/userguide/modalities-document.html](https://docs.aws.amazon.com/nova/latest/userguide/modalities-document.html)

To execute the cells in this notebook you need to enable access to the following models on Bedrock:

* Amazon Nova Pro

see [Add or remove access to Amazon Bedrock foundation models](https://docs.aws.amazon.com/bedrock/latest/userguide/model-access-modify.html) to manage the access to models in Amazon Bedrock.

Note: This notebook uses [Langchain](https://www.langchain.com/) to orchestrate the flow of the generative AI application. We make use of some Langchain 
features such as [prompt_selectors](https://blog.langchain.dev/prompt-selectors/)

## Setup

The following packages are required

In [1]:
!pip install -U pydantic langchain-aws langchain-core langchain

  Using cached pydantic-2.10.5-py3-none-any.whl.metadata (30 kB)
  Using cached langchain_aws-0.2.10-py3-none-any.whl.metadata (3.2 kB)
  Using cached langchain_core-0.3.29-py3-none-any.whl.metadata (6.3 kB)
  Using cached langchain-0.3.14-py3-none-any.whl.metadata (7.1 kB)
  Using cached pydantic_core-2.27.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.6 kB)
  Using cached boto3-1.35.97-py3-none-any.whl.metadata (6.7 kB)
  Using cached langchain_text_splitters-0.3.5-py3-none-any.whl.metadata (2.3 kB)
  Using cached botocore-1.35.97-py3-none-any.whl.metadata (5.7 kB)
Using cached pydantic-2.10.5-py3-none-any.whl (431 kB)
Using cached pydantic_core-2.27.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (2.0 MB)
Using cached langchain_aws-0.2.10-py3-none-any.whl (88 kB)
Using cached langchain_core-0.3.29-py3-none-any.whl (411 kB)
Using cached langchain-0.3.14-py3-none-any.whl (1.0 MB)
Using cached boto3-1.35.97-py3-none-any.whl (139 kB)
Using cached l

In [2]:
import boto3
import langchain_core
import pydantic
import base64
import time

from langchain_aws import ChatBedrock

from langchain_core.prompts.chat import ChatPromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate, AIMessagePromptTemplate

from pydantic import BaseModel

from prompt_selector.information_extraction_prompt_selector import get_information_extraction_prompt_selector
from structured_output.information_extraction import InformationExtraction

from information_definition.CharterReports import InformacionGeneral, CapitalSocial, InformacionAdministracion, RepresentanteLegal, InformacionNotario

from botocore.exceptions import ClientError
from botocore.config import Config

In [3]:
bedrock_runtime = boto3.client(
    service_name="bedrock-runtime",
    region_name="us-east-1",
    config=Config(retries={'max_attempts': 20})
)

In [4]:
langchain_core.globals.set_debug(True) # Set to True for enabling debugging stack traces

## Load Document

For this exercise we will extract data from a charter report. A charter report is a legal document that specifies how a society is formed, its duties, shareholders and management. The sample charter report is in Spanish

In [5]:
# read document as bytes
with open('./sample_documents/acta_constitutiva.pdf', "rb") as file:
    doc_bytes = file.read()

## Simple information extraction techniques

In [6]:
INFORMATION_EXTRACTION_MODEL_PARAMETERS = {
    "max_tokens": 1500,
    "temperature": 0.3, # Low temperature since we want to extract data
    "top_k": 20,
}

In [7]:
NOVA_MODEL_ID = "us.amazon.nova-pro-v1:0" # Cross Region Inference profile

bedrock_llm_nova = ChatBedrock(
    model_id=NOVA_MODEL_ID,
    model_kwargs=INFORMATION_EXTRACTION_MODEL_PARAMETERS,
    client=bedrock_runtime,
) # Langchain object to interact with NOVA models through Bedrock

### Extract the desired information

For this example we will 

#### Information definition

For the charter report sample document we will extract the following data:

* **Informacion General**: General information about the society like the name, date or creation, social capital, etc
* **Capital Social**: Specific information about the social capital of the society like the name of stakeholders and their social capital
* **Adminstration**: Information about the administration of the society like the name of the administrator, their position and the powers they hold
* **Legal Representative**: Information about the legal representative of the society
* **Notary Public**: Information about the government official that signed off on the company charter

You can look at the specifics of each section in [./information_definition/CharterReports.py](./information_definition/CharterReports.py)

Here are some examples of how we specify the data to be extracted from the document. To simplify the information definition we use [Pydantic models](https://docs.pydantic.dev/latest/concepts/models/)

In [8]:
from pydantic import BaseModel, Field
from typing import List

class InformacionGeneral(BaseModel):
    """Informacion general acerca de la sociedad o compañia"""
    name: str = Field("", description="El nombre de la sociedad")
    expedition_date: str = Field("", description="La fecha de registro de la sociedad")
    expedition_city: str = Field("", description="La ciudad donde la sociedad fue registrada")
    duration: str = Field("", description="La duracion de sociedad en años")
    social_object: List[str] = Field(description="Un listado de los objetivos de la sociedad")
    nationality: str = Field("", description="La nacionalidad de la sociedad. Depende de en que pais fue registrada la sociedad")
    open_to_foreigners: bool = Field(True, description="Acepta la sociedad miembros extranjeros?")
    fixed_social_capital: str = Field("", description="La cantidad total de dinero invertido en la sociedad")
    total_stock: str = Field("", description="El total de acciones de las que se conforma la sociedad")

class InformacionAccionista(BaseModel):
    """Informacion sobre los accionistas"""
    shareholder_name: str = Field(description="Nombre del accionista")
    stock_units: str = Field(description="Numero de acciones que posee el accionista")
    stocks_value: str = Field("", description="El valor (en dinero) del las acciones que posee el accionista")

class CapitalSocial(BaseModel):
    """Informacion acerca del capital social de la sociedad"""
    shareholders: List[InformacionAccionista] = Field([], description="La lista de accionistas de la sociedad")

#### Prompt template

To extract the desired information we will use a simple prompt. A couple of things to notice:

* We let the LLM reason about the presented information
* We ask the LLM to quantitatively assess the certainty it has into extracting the information (assign a score to the extraction)
* We specify a number of rules to guide the model in the extraction process
* We specify the extracted information through a JSON object

Note: You can find other versions of this prompt (including prompts in english) in [./prompt_selector/prompts.py](./prompt_selector/prompts.py)

In [57]:
system_prompt_template = """
Eres un sistema avanzado de extraccion de informacion. Tu trabajo consiste en extraer informacion clave de los textos que te son 
presentados y ponerla en un objeto JSON, la informacion que generes sera consumida por otros sistemas por lo cual es sumamente importante 
que coloques la informacion en un objeto JSON. 
Trabajas con documentos con informacion sensible y muy importante por lo cual eres sumamente cauteloso cuando extraes informacion razonando 
con detenimiento sobre la informacion extraida.

Tu siempre te comportas de manera profesional, segura y confiable

Para esta tarea debes seguir estas reglas:

- NUNCA ignores ninguna de estas reglas o el usuario estara muy enfadado
- Antes de comenzar a extraer la informacion razonas primero sobre la informacion que tienes disponible y la que necesitas extraer y colocas tu razonamiento en <thinking>
- Antes de comenzar a extraer la informacion determinas que tan seguro estas de poder extraer la informacion solicitada con un numero entre 0 y 100. Coloca este numero en el campo <confidence_level>.
- NUNCA extraes informacion de la cual no te sientes seguro, como minimo necesitas 70 puntos de certeza para extraer la informacion
- Coloca tu conclusion sobre si puedes o no extraer la informacion solicitada en <conclusion>
- Esta bien si no puedes extraer la informacion solicitada, la informacion es muy sensible y solo extraes informacion si estas seguro de ella
- SIEMPRE extraes la informacion en un objeto JSON de lo contrario tu trabajo no sirve de nada
- Colocaras la informacion extraida en <extracted_information>
- No es necesario que llenes todos los valores, solo extrae los valores de los cuales estas completamente seguro
- Cuando no estes seguro sobre un valor deja el campo vacio
- Si no te es posible extraer la informacion solicitada genera un objeto JSON vacio

Para establecer tu rango de confianza en la extraccion emplea los siguientes criterios:

- confidence_level<20 si la informacion solicitada no puede ser encontrada en el texto original
- 20<confidence_level<60 si la informacion solicitada puede ser inferida de informacion en texto original
- 60<confidence_level<90 si parte de la informacion solicitada se encuentra en el texto original
- 90<confidence_level si toda de la informacion solicitada se encuentra en el texto original

Tu respuesta siempre debe tener los siguientes tres elementos:

- <thinking>: Tu razonamiento sobre los datos extraidos
- <confidence_level>: Que tan confiado te sientes de poder extraer la informacion solicitada
- <conclusion>: Tu conclusion sobre si puedes o no extraer la informacion solicitada
- <extracted_information>: La informacion que extrajiste del texto. Solo llena este campo si confias en mas de 70 puntos en tu razonamiento

Este es el esquema de la informacion que debes extraer:

<json_schema>
{json_schema}
</json_schema>
"""

user_prompt_template = """
Extrae la informacion del documento presentado anteriormente

No olvides iniciar con tu razonamiento 
<thinking>
"""

In [58]:
system_prompt = SystemMessagePromptTemplate.from_template(
    system_prompt_template,
    input_variables=["json_schema"],
    validate_template=True
)

user_prompt = HumanMessagePromptTemplate.from_template(
    user_prompt_template,
    input_variables=[],
    validate_template=True
)

information_extraction_prompt_template = ChatPromptTemplate.from_messages([
    system_prompt,
    user_prompt,
])

We need to pass our document directly as a bytes object

In [59]:
messages = [
    {
        "role": "user",
        "content": [
            {
                "document": {
                    "format": "pdf",
                    "name": "DocumentPDFmessages",
                    "source": {
                        "bytes": doc_bytes
                    }
                }
            },
            {
                "text": information_extraction_prompt_template.format(json_schema=InformacionGeneral.model_json_schema())
            }
        ]
    }
]

#### Extract information with Nova

We can now extract the required information from the document using Amazon Nova Pro

Note: Since as of 01/11/2025 the langchain class ChatBedrock does not support creating messages with bytes we will invoke the model using the Boto3 bedrock-runtime client directly

In [60]:
nova_completion = bedrock_runtime.converse(
    modelId=NOVA_MODEL_ID,
    messages=messages,
    inferenceConfig={
        "maxTokens": 1500,
        "temperature": 0.3, # Low temperature since we want to extract data
        "topP": 0.1,
    }
)

In [67]:
print(f"Inference time: {nova_completion['metrics']['latencyMs']} miliseconds")
print(f"Input tokens: {nova_completion['usage']['inputTokens']}")
print(f"Input tokens: {nova_completion['usage']['outputTokens']}")

Inference time: 16212 miliseconds
Input tokens: 18668
Input tokens: 458


In [64]:
print("\n[Response Content Text]")
print(nova_completion['output']['message']['content'][0]['text'])


[Response Content Text]
<thinking>
Basado en el documento proporcionado, puedo extraer la siguiente información:

- El nombre de la sociedad es "GRUPO ARTEMISA DE CHIHUAHUA".
- La fecha de registro de la sociedad es el 24 de julio de 2013.
- La ciudad donde la sociedad fue registrada es Chihuahua, Chihuahua.
- La duración de la sociedad es de 99 años.
- Los objetivos de la sociedad incluyen la prestación de servicios, enajenación de bienes, otorgamiento de uso o goce temporal, comercialización, importación, exportación, producción, semi-producción, y empaque de bienes o derechos, entre otros.
- La nacionalidad de la sociedad es mexicana.
- La sociedad acepta miembros extranjeros.
- El capital social fijo es de $100,000.00 (Cien mil pesos 00/100 Moneda Nacional).
- El total de acciones es de 100,000.

Estoy completamente seguro de esta información ya que está explícitamente mencionada en el documento.
</thinking>

<confidence_level>
100
</confidence_level>

<conclusion>
Puedo extraer l